In [1]:
import numpy as np
import pandas as pd

In [11]:
data = np.loadtxt('3d_ebsd_20240619_combined_data.txt', skiprows=1)
print(type(data))
#print(data[:20])

<class 'numpy.ndarray'>


In [13]:
def euler_to_orientation(euler1, euler2, euler3, phase):
    if phase == 0:
        # If phase is 0, set R to all zeros
        R = np.zeros((3, 3))
        quaternion = np.array([0, 0, 0, 0])
    else:
        # Convert Euler angles to rotation matrix
        R = np.array([
            [np.cos(euler1) * np.cos(euler3) - np.sin(euler1) * np.sin(euler3) * np.cos(euler2),
             np.sin(euler1) * np.cos(euler3) + np.cos(euler1) * np.sin(euler3) * np.cos(euler2),
             np.sin(euler3) * np.sin(euler2)],
            [-np.cos(euler1) * np.sin(euler3) - np.sin(euler1) * np.cos(euler3) * np.cos(euler2),
             -np.sin(euler1) * np.sin(euler3) + np.cos(euler1) * np.cos(euler3) * np.cos(euler2),
             np.cos(euler3) * np.sin(euler2)],
            [np.sin(euler1) * np.sin(euler2),
             -np.cos(euler1) * np.sin(euler2),
             np.cos(euler2)]
        ])

        # Convert rotation matrix to quaternion
        q0 = 0.5 * np.sqrt(1 + R[0, 0] + R[1, 1] + R[2, 2])
        q1 = (R[2, 1] - R[1, 2]) / (4 * q0)
        q2 = (R[0, 2] - R[2, 0]) / (4 * q0)
        q3 = (R[1, 0] - R[0, 1]) / (4 * q0)
        
        # Convert quaternion to a NumPy array
        quaternion = np.array([q0, q1, q2, q3])

    return R, quaternion

In [15]:
def process_data(data):
    num_rows = len(data)
    results = np.empty((num_rows, 13))  # 9 elements for R and 4 elements for quaternion
    for i, row in enumerate(data):
        phase, euler1, euler2, euler3 = row[0], np.deg2rad(row[6]), np.deg2rad(row[7]), np.deg2rad(row[8])  # Convert degrees to radians
        R, quaternion = euler_to_orientation(phase, euler1, euler2, euler3)
        results[i] = np.concatenate((R.ravel(), quaternion.ravel()))  # Concatenate the arrays and flatten
    return results

results = process_data(data)
selected_columns = data[:, [0,1,2,3,6,7,8]]  # Select the Phase, x, y, z, Euler1, Euler2, Euler3 columns
appended_data = np.hstack((selected_columns, results))

# Save the results as a .txt file
header = 'Phase\tx\ty\tz\tEuler1\tEuler2\tEuler3\t' + '\t'.join([f'R{i+1}' for i in range(9)]) + '\t' + '\t'.join([f'q{i+1}' for i in range(4)])
np.savetxt('3d_ebsd_20240619_R_appended_data', appended_data, delimiter='\t', header=header, fmt='%s')

In [64]:
#print(len(appended_data))

17987028
